# Refactoring a cura di Giulio, Jacopo e Mohammed



Installiamo le librerie necessarie:

In [ ]:
# Librerie da installare
install.packages(c('httr', 'readr', 'data.table', 'psych', 'htmltab', 'stringi', 'bizdays', 'stringr', 'dplyr', 'lubridate', 'zoo', 'reshape2'))

# Parte 1 - REST API

## Consuming APIs and JSON

In [6]:
library(httr)

WScall <- function(endpoint, parameters) {
    result <- GET(endpoint, query = parameters)
    return(result)
}

ad <- "1600 Pennsylvania Avenue, Washington, DC"
urla <- "http://nominatim.openstreetmap.org/search"
paramA <- list(q = ad, addressdetails = 1, format = "json")

resA <- WScall(urla, paramA)

if (resA$status_code == 200) {
    adrjson <- content(resA, as = "parsed")
    lat = adrjson[[1]]$lat
    lon = adrjson[[1]]$lon
}

urlw <- "http://forecast.weather.gov/MapClick.php"
paramW <- list(lat = lat, lon = lon, FcstType = "json")

resW <- WScall(urlw, paramW)

if (resW$status_code == 200) {
    weajson <- content(resW, as = "parsed")
    weajson$currentobservation
}

$id
[1] "KDCA"

$name
[1] "Washington/Reagan National Airport, DC"

$elev
[1] "13"

$latitude
[1] "38.85"

$longitude
[1] "-77.03"

$Date
[1] "22 Jun 03:52 am EDT"

$Temp
[1] "77"

$Dewp
[1] "71"

$Relh
[1] "82"

$Winds
[1] "6"

$Windd
[1] "220"

$Gust
[1] "0"

$Weather
[1] "Mostly Cloudy"

$Weatherimage
[1] "nbkn.png"

$Visibility
[1] "10.00"

$Altimeter
[1] "1015.1"

$SLP
[1] "29.98"

$timezone
[1] "EDT"

$state
[1] "VA"

$WindChill
[1] "NA"

# Parte 2 - Libreria PANDAS

## Nouns (objects) in Pandas
### Data Frames
Con due vettori creo il dataframe

In [7]:
number = c(1, 2, 3)
animal = c('cat', 'dog', 'mouse')
df1 = data.frame(number, animal)
df1

df1$animal

number,animal
1,cat
2,dog
3,mouse


[1] cat   dog   mouse
Levels: cat dog mouse

1)Richiamo la colonna "animal" del data frame.

2)Estraggo gli elementi della colonna "animal" come una lista.

3)Per creare un data frame possiamo anche operare in altri modi.

4)Con la libreria compare posso confrontare i due data frame.

In [8]:
df1["animal"] #1

as.list(df1$animal) #2

df2 <- data.frame(animal=animal, number= number) #3
df2
df2 <- as.data.frame(cbind(animal,number)) #3
df2

library(compare)   #4
compare(df1,df2,allowAll=TRUE)

animal
cat
dog
mouse


[[1]]
[1] cat
Levels: cat dog mouse

[[2]]
[1] dog
Levels: cat dog mouse

[[3]]
[1] mouse
Levels: cat dog mouse

animal,number
cat,1
dog,2
mouse,3


animal,number
cat,1
dog,2
mouse,3


ERROR: Error in library(compare): there is no package called 'compare'


# Verbs (operations) in Pandas
## Loading data (and basic statistics / visualization)

In [9]:
library(readr)
library(data.table)
names = c("State_Code", "County_Code", "Census_Tract_Number", "NUM_ALL", "NUM_FHA", "PCT_NUM_FHA", "AMT_ALL", "AMT_FHA", "PCT_AMT_FHA")
dt = as.data.table(read_csv("C:/Users/GiulioVannini/Documents/Visual Studio 2017/Projects/MABIDA2017/Gigli/Management science/Data/fha_by_tract.csv", col_names = names))
head(dt)

#inseriamo una nuova colonna GEOID nel dataframe

dt$GEOID = as.character(with(dt, as.numeric(Census_Tract_Number) * 100 + 10 ^ 6 * as.numeric(County_Code) + 10 ^ 9 * as.numeric(State_Code)  ))
head(dt)

df$GEOID = NULL # per escludere la colonna... ma perche' dovremmo dropparla?

df = dt[-1,]

# Indici su data frame
setkey(dt, State_Code, County_Code)
head(dt)
data.frame(unclass(summary(dt)))

str(dt)

library(psych)
describeBy(dt, dt$PCT_AMT_FHA)
describeBy(dt)
hist(dt$PCT_AMT_FHA, col = rgb(1, 0, 0, 0.5))

#Creo una nuova colonna applicando il logaritmo ai valori di AMT_ALL del data frame

dt$LOG_AMT_ALL = log1p(df$AMT_ALL)
hist(dt$LOG_AMT_ALL, col = rgb(1, 0, 0, 0.5))

Parsed with column specification:
cols(
  State_Code = col_character(),
  County_Code = col_character(),
  Census_Tract_Number = col_character(),
  NUM_ALL = col_integer(),
  NUM_FHA = col_integer(),
  PCT_NUM_FHA = col_integer(),
  AMT_ALL = col_integer(),
  AMT_FHA = col_integer(),
  PCT_AMT_FHA = col_integer()
)
Warning message:
"115232 parsing failures.
 row         col               expected actual
1217 PCT_NUM_FHA no trailing characters  .9091
1217 PCT_AMT_FHA no trailing characters  .7336
1218 PCT_NUM_FHA no trailing characters  .3333
1218 PCT_AMT_FHA no trailing characters  .6689
1219 PCT_NUM_FHA no trailing characters  .9091
.... ........... ...................... ......
See problems(...) for more details.
"

State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA
08,075,NA,1,1,100,258,258,100
28,049,0103.01,1,1,100,71,71,100
40,003,NA,1,1,100,215,215,100
39,113,0603.00,3,3,100,206,206,100
12,105,0124.04,2,2,100,303,303,100
12,086,9808.00,1,1,100,188,188,100


State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
08,075,NA,1,1,100,258,258,100,NA
28,049,0103.01,1,1,100,71,71,100,28049010301
40,003,NA,1,1,100,215,215,100,NA
39,113,0603.00,3,3,100,206,206,100,39113060300
12,105,0124.04,2,2,100,303,303,100,12105012404
12,086,9808.00,1,1,100,188,188,100,12086980800


ERROR: Error in df$GEOID = NULL: object of type 'closure' is not subsettable


# Indexing data frames

In [10]:
head(dt$State_Code) #ottengo i valori della colonna

head(dt[,c(State_Code, County_Code)])  #opero su sottoinsieme del dataframe
head(dt[, County_Code, keyby = State_Code])
str(df$State_Code)

dt[12550,]

dt[12550, 'State_Code'] #riferimenti misti

dt[12545:12550, County_Code, keyby = State_Code]

dt[3, 5]

dt[3:5, 2:4]

[1] "08" "28" "40" "39" "12" "12"

[1] "08" "28" "40" "39" "12" "12"

State_Code,County_Code
NA,NA
01,091
01,091
01,063
01,097
01,073


ERROR: Error in df$State_Code: object of type 'closure' is not subsettable


# Filtering data

In [11]:
head(subset(dt, State_Code == 33)) #Creo un sottoindieme del data frame per valori di State_Code corrispondenti a 33.
head(subset(dt, (State_Code == 33) | (Census_Tract_Number == 9613))) #Sottoinsieme con State_Code=33 | Census_Tract_Number=9613
dt[State_Code == 33]

State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
33,011,0013.00,13,9,NA,1700,1310,NA,33011001300
33,011,0106.00,32,20,NA,5031,3510,NA,33011010600
33,011,0024.00,72,45,NA,10435,7040,NA,33011002400
33,013,0440.00,24,14,NA,3479,2234,NA,33013044000
33,011,0002.02,10,6,60,1574,993,NA,33011000202
33,011,0018.00,35,21,60,5222,3293,NA,33011001800


State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
33,011,0013.00,13,9,NA,1700,1310,NA,33011001300
33,011,0106.00,32,20,NA,5031,3510,NA,33011010600
33,011,0024.00,72,45,NA,10435,7040,NA,33011002400
33,013,0440.00,24,14,NA,3479,2234,NA,33013044000
33,011,0002.02,10,6,60,1574,993,NA,33011000202
33,011,0018.00,35,21,60,5222,3293,NA,33011001800


State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
33,011,0013.00,13,9,NA,1700,1310,NA,33011001300
33,011,0106.00,32,20,NA,5031,3510,NA,33011010600
33,011,0024.00,72,45,NA,10435,7040,NA,33011002400
33,013,0440.00,24,14,NA,3479,2234,NA,33013044000
33,011,0002.02,10,6,60,1574,993,NA,33011000202
33,011,0018.00,35,21,60,5222,3293,NA,33011001800
33,011,0007.00,37,21,NA,6821,4205,NA,33011000700
33,015,0035.00,35,20,NA,5132,3101,NA,33015003500
33,013,0443.00,29,16,NA,4385,2571,NA,33013044300
33,013,0322.00,13,7,NA,1706,991,NA,33013032200


# Joining data

In [12]:
dt2 = as.data.table(read_tsv("C:/Users/GiulioVannini/Documents/Visual Studio 2017/Projects/MABIDA2017/Gigli/Management science/Data/2013_Gaz_tracts_national.tsv", col_names = T))
head(dt2)
dt$GEOID = as.character(with(dt, as.numeric(Census_Tract_Number) * 100 + 10 ^ 6 * as.numeric(County_Code) + 10 ^ 9 * as.numeric(State_Code)))
dt_joined = dt[dt2, on = "GEOID"]
head(dt_joined)

Parsed with column specification:
cols(
  USPS = col_character(),
  GEOID = col_character(),
  ALAND = col_integer(),
  AWATER = col_integer(),
  ALAND_SQMI = col_double(),
  AWATER_SQMI = col_double(),
  INTPTLAT = col_double(),
  INTPTLONG = col_double()
)
Warning message:
"698 parsing failures.
 row    col   expected      actual
1182 ALAND  an integer 18082854844
1182 AWATER an integer 20793145269
1183 ALAND  an integer 8547315155 
1183 AWATER an integer 22088655933
1184 ALAND  an integer 2823181227 
.... ...... .......... ...........
See problems(...) for more details.
"

USPS,GEOID,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
AL,01001020100,9809939,36312,3.788,0.014,32.48179,-86.49025
AL,01001020200,3340498,5846,1.290,0.002,32.47576,-86.47247
AL,01001020300,5349274,9054,2.065,0.003,32.47402,-86.45970
AL,01001020400,6382705,16244,2.464,0.006,32.47103,-86.44484
AL,01001020500,11397734,48412,4.401,0.019,32.45892,-86.42182
AL,01001020600,8020363,60048,3.097,0.023,32.44737,-86.47683


State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID,USPS,ALAND,AWATER,ALAND_SQMI,AWATER_SQMI,INTPTLAT,INTPTLONG
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020100,AL,9809939,36312,3.788,0.014,32.48179,-86.49025
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020200,AL,3340498,5846,1.290,0.002,32.47576,-86.47247
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020300,AL,5349274,9054,2.065,0.003,32.47402,-86.45970
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020400,AL,6382705,16244,2.464,0.006,32.47103,-86.44484
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020500,AL,11397734,48412,4.401,0.019,32.45892,-86.42182
NA,NA,NA,NA,NA,NA,NA,NA,NA,01001020600,AL,8020363,60048,3.097,0.023,32.44737,-86.47683


# Aggregating data

In [13]:
usps_groups = group_by(dt_joined, USPS)
usps_groups
group_AK_5 <- filter(usps_groups, USPS == 'AK')[5,] #filtro

df_by_state <- summarise(usps_groups, count = n(), AMT_FHA = sum(AMT_FHA), AMT_ALL = sum(AMT_ALL), NUM_FHA = sum(NUM_FHA), NUM_ALL = sum(NUM_ALL))
head(df_by_state)

df_by_state$PCT_AMT_FHA <- 100.0 * df_by_state$AMT_FHA / df_by_state$AMT_ALL

hist(df_by_state$PCT_AMT_FHA, breaks = 20)

df_by_state2 <- summarise(usps_groups, count = n(), sum_NUM_FHA = sum(NUM_FHA), mean_NUM_ALL = mean(NUM_ALL))
head(df_by_state2)
dplyr::arrange(usps_groups, desc(INTPTLAT))[1,]
names(usps_groups)

#Funzione per cui, per ogni valore di USPS ottengo il massimo di INTPTLAT               
farthest_north <- function(state_df) {
    result <- dplyr::arrange(state_df, desc(INTPTLAT))[1,]
    return(result)
}

#Creo un dataframe che mi restituisce il valore massimo trovato nella colonna INTPTLAT per valore univoco di USPS 
northest <- summarise(usps_groups, farthest_north = max(INTPTLAT))
northest

ERROR: Error in eval(expr, envir, enclos): non trovo la funzione "group_by"


# Sorting by indices and columns

In [14]:
dtbystate = dt[order(State_Code)]
dtbystate
dtbyAMTFHA = dt[order(AMT_FHA)]
dtbyAMTFHA

State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
01,091,9733.00,1,1,100,214,214,100,1091973300
01,091,9732.00,2,2,100,168,168,100,1091973200
01,063,0600.00,1,1,100,141,141,100,1063060000
01,097,0040.00,1,1,100,25,25,100,1097004000
01,073,0008.00,1,1,100,58,58,100,1073000800
01,097,0027.00,1,1,100,60,60,100,1097002700
01,097,0049.00,1,1,100,70,70,100,1097004900
01,025,9575.00,2,2,100,206,206,100,1025957500
01,097,0032.04,3,3,100,201,201,100,1097003204
01,099,0762.00,2,2,100,193,193,100,1099076200


State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
21,175,9502.00,2,0,0,271,0,0,21175950200
32,003,0024.04,1,0,0,79,0,0,32003002404
36,061,0183.00,51,0,0,33258,0,0,36061018300
51,710,0040.01,16,0,0,4663,0,0,51710004001
42,131,NA,1,0,0,127,0,0,NA
22,093,0405.00,1,0,0,30,0,0,22093040500
37,063,0015.01,1,0,0,160,0,0,37063001501
72,071,4103.00,4,0,0,521,0,0,72071410300
37,115,NA,3,0,0,452,0,0,NA
22,033,0052.00,5,0,0,704,0,0,22033005200


# Unique values

In [15]:
head(unique(dt))
nrow(unique(dt))

State_Code,County_Code,Census_Tract_Number,NUM_ALL,NUM_FHA,PCT_NUM_FHA,AMT_ALL,AMT_FHA,PCT_AMT_FHA,GEOID
08,075,NA,1,1,100,258,258,100,NA
28,049,0103.01,1,1,100,71,71,100,28049010301
40,003,NA,1,1,100,215,215,100,NA
39,113,0603.00,3,3,100,206,206,100,39113060300
12,105,0124.04,2,2,100,303,303,100,12105012404
12,086,9808.00,1,1,100,188,188,100,12086980800


[1] 72035

# Handling missing and NA data

In [16]:
library(zoo)
dt[, c('GEOID')][1:10]
is.na(dt[, c('GEOID')])[1:10]
length(dt[, c('GEOID')])
length(na.omit(dt[, c('GEOID')]))
dt$FILL0 <- dt$GEOID
dt$FILL0[which(is.na(dt$GEOID))] <- 0 #sostituisco Na con valore 0
dt$FILL_mean <- dt$GEOID
dt$FILL_mean[which(is.na(dt$GEOID))] <- mean(dt$GEOID, na.rm = TRUE)
dt$FILL_inter <- dt$GEOID
dt$FILL_inter <- na.approx(dt$FILL_inter)
dt[, c('GEOID', 'FILL0', 'FILL_mean', 'FILL_inter')][1:10,]

Warning message:
"package 'zoo' was built under R version 3.3.3"
Attaching package: 'zoo'

The following objects are masked from 'package:base':

    as.Date, as.Date.numeric



GEOID
NA
28049010301
NA
39113060300
12105012404
12086980800
39035120200
12103020700
36119003000
NA


[1]  TRUE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE

[1] 1

[1] 1

Warning message in mean.default(dt$GEOID, na.rm = TRUE):
"argument is not numeric or logical: returning NA"Warning message in `[<-.data.table`(x, j = name, value = value):
"Supplied 72034 items to be assigned to 72035 items of column 'FILL_inter' (recycled leaving remainder of 1 items)."Warning message in `[<-.data.table`(x, j = name, value = value):
"Coerced 'double' RHS to 'character' to match the column's type; may have truncated precision. Either change the target column to 'double' first (by creating a new 'double' vector length 72035 (nrows of entire table) and assign that; i.e. 'replace' column), or coerce RHS to 'character' (e.g. 1L, NA_[real|integer]_, as.*, etc) to make your intent clear and for speed. Or, set the column type correctly up front when you create the table and stick to it, please."

GEOID,FILL0,FILL_mean,FILL_inter
NA,0,NA,28049010301
28049010301,28049010301,28049010301,33581035300.5
NA,0,NA,39113060300
39113060300,39113060300,39113060300,12105012404
12105012404,12105012404,12105012404,12086980800
12086980800,12086980800,12086980800,39035120200
39035120200,39035120200,39035120200,12103020700
12103020700,12103020700,12103020700,36119003000
36119003000,36119003000,36119003000,42116005852
NA,0,NA,48113008704


# Manipulating strings

In [17]:
library(stringr)
library(dplyr)
dtstring = dt_joined %>% filter(str_detect(USPS, "A"))

Warning message:
"package 'stringr' was built under R version 3.3.3"Warning message:
"package 'dplyr' was built under R version 3.3.3"------------------------------------------------------------------------------
data.table + dplyr code now lives in dtplyr.
Please library(dtplyr)!
------------------------------------------------------------------------------

Attaching package: 'dplyr'

The following objects are masked from 'package:data.table':

    between, first, last

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



# Indices in Pandas

In [18]:

s1 <- c(1, 2, 3)
names(s1) = c('a', 'b', 'c')
s2 <- c(3, 2, 1)
names(s2) = c('c', 'b', 'a')
s1 + s2
s3 <- c(3, 2, 1)
names(s3) = c('c', 'd', 'e')
s1 + s3
append(s1, s3)

a b c 
4 4 4

a b c 
4 4 4

a b c c d e 
1 2 3 3 2 1

# Function application and mapping

In [19]:
dff <- as.data.frame(matrix(1:24, ncol = 6, nrow = 4, byrow = TRUE))
head(sin(dff))
dff
apply(dff, 1:2, function(x) sprintf("%.2f", x))
apply(dff, 2, function(x) max(x) - min(x))
apply(dff, 1, function(x) max(x) - min(x))

V1,V2,V3,V4,V5,V6
0.8414710,0.9092974,0.1411200,-0.756802495,-0.9589243,-0.2794155
0.6569866,0.9893582,0.4121185,-0.544021111,-0.9999902,-0.5365729
0.4201670,0.9906074,0.6502878,-0.287903317,-0.9613975,-0.7509872
0.1498772,0.9129453,0.8366556,-0.008851309,-0.8462204,-0.9055784


V1,V2,V3,V4,V5,V6
1,2,3,4,5,6
7,8,9,10,11,12
13,14,15,16,17,18
19,20,21,22,23,24


V1,V2,V3,V4,V5,V6
1.00,2.00,3.00,4.00,5.00,6.00
7.00,8.00,9.00,10.00,11.00,12.00
13.00,14.00,15.00,16.00,17.00,18.00
19.00,20.00,21.00,22.00,23.00,24.00


V1 V2 V3 V4 V5 V6 
18 18 18 18 18 18

[1] 5 5 5 5

# Pandas HTML data import example   

In [20]:
library(htmltab)
library(stringi)
require(data.table)
url <- "http://en.wikipedia.org/wiki/List_of_tallest_buildings_and_structures_in_the_world"
tallest <- htmltab(doc = url, which = 3)
tl = t(as.data.table(stri_extract_all(tallest$Coordinates, regex = "-?\\d{1,3}+\\.?\\d{4,6}")))
tallest$Latitude = tl[, 3]
tallest$Longitude = tl[, 4]
rm(tl)
head(tallest)

Warning message:
"package 'htmltab' was built under R version 3.3.3"

,Category,Structure,Country,City,Height (metres),Height (feet),Year built,Coordinates,Latitude,Longitude
2,Mixed use,Burj Khalifa,Â United Arab Emirates,Dubai,828.1,"2,717",2010,25Â°11â€²50.0â€³N55Â°16â€²26.6â€³Eï»¿ / ï»¿25.197222Â°N 55.274056Â°E,25.197222,55.274056
3,Self-supporting tower,Tokyo Skytree,Â Japan,Tokyo,634,"2,080",2011,35Â°42â€²36.5â€³N139Â°48â€²39â€³Eï»¿ / ï»¿35.710139Â°N 139.81083Â°E,35.710139,139.81083
4,Guyed steel lattice mast,KVLY-TV mast,Â United States,"Blanchard, North Dakota",628.8,"2,063",1963,47Â°20â€²32â€³N97Â°17â€²25â€³Wï»¿ / ï»¿47.34222Â°N 97.29028Â°W,47.34222,97.29028
5,Clock building,Abraj Al Bait Towers,Â Saudi Arabia,Mecca,601,"1,972",2011,21Â°25â€²08â€³N39Â°49â€²35â€³Eï»¿ / ï»¿21.41889Â°N 39.82639Â°E,21.41889,39.82639
6,Office,One World Trade Center,Â United States,"New York, NY",541,"1,776",2013,40Â°42â€²46.8â€³N74Â°0â€²48.6â€³Wï»¿ / ï»¿40.713000Â°N 74.013500Â°W,40.713000,74.013500
7,Military structure,Large masts of INS Kattabomman,Â India,Tirunelveli,471,"1,545",2014,8Â°22â€²42.52â€³N77Â°44â€²38.45â€³Eï»¿ / ï»¿8.3784778Â°N 77.7440139Â°EÂ ; 8Â°22â€²30.13â€³N77Â°45â€²21.07â€³Eï»¿ / ï»¿8.3750361Â°N 77.7558528Â°E,8.375036,77.755852


# Pandas Timestamps

In [21]:
Sys.setlocale("LC_TIME", "C")
as.POSIXlt("July 4, 2016", format = "%B %d, %Y")
as.POSIXlt('Monday, July 4, 2016', format = "%A, %B %d, %Y")
as.POSIXlt('Tuesday, July 4th, 2016', format = "%A, %B %dth, %Y")
as.POSIXlt('Monday, July 4th, 2016 05:00 PM', format = "%A, %B %dth, %Y %I:%M %p")
as.POSIXlt('04/07/2016T17:20:13.123456', format = "%d/%m/%YT%H:%M:%OS")
as.Date(as.POSIXlt(1467651600000000000 / 1000000000, origin = "1970-01-01"))

july4 = as.POSIXct('Monday, July 4th, 2016 05:00 PM', format = "%A, %B %dth, %Y %I:%M %p", tz = "US/Eastern")
labor_day = as.POSIXct('9/5/2016 12:00', format = "%d/%m/%Y %H:%M", tz = "US/Eastern")
thanksgiving = as.POSIXct('11/24/2016 16:00', format = "%m/%d/%Y %H:%M")

labor_day - july4
library(bizdays)
library(lubridate)
last_day <- function(date) {
    ceiling_date(date, "month") - days(1)
}
last_day(ymd(20160704))
create.calendar(name = 'ANBIMA', holidays = holidaysANBIMA, weekdays = c('saturday', 'sunday'))
bizdays.options$set(default.calendar = 'ANBIMA')
cal = bizdays.options$get("default.calendar")
bizdays::offset(july4, 5, cal)
bizdays::offset(july4, -1, cal)
bizdays::offset(last_day(ymd(20160704)), 0, cal) # last business day of the month.

require(bizdays)
business_days = bizseq('2016-01-01', '2016-12-31', "ANBIMA")
business_days

dtimed = data.table(x = business_days, y = seq(1, length(business_days)))
setkey(dtimed, x)

d <- c("2009-03-07 12:00", "2009-03-08 12:00", "2009-03-28 12:00", "2009-03-29 12:00", "2009-10-24 12:00", "2009-10-25 12:00", "2009-10-31 12:00", "2009-11-01 12:00")
t1 <- as.POSIXct(d, "America/Los_Angeles")
cbind(US = format(t1), UK = format(t1, tz = "Europe/London"))

[1] "C"

[1] "2016-07-04 CEST"

[1] "2016-07-04 CEST"

[1] "2016-07-04 CEST"

[1] "2016-07-04 17:00:00 CEST"

...

[1] "2016-07-11"

[1] "2016-07-01"

[1] NA

  [1] "2016-01-04" "2016-01-05" "2016-01-06" "2016-01-07" "2016-01-08"
  [6] "2016-01-11" "2016-01-12" "2016-01-13" "2016-01-14" "2016-01-15"
 [11] "2016-01-18" "2016-01-19" "2016-01-20" "2016-01-21" "2016-01-22"
 [16] "2016-01-25" "2016-01-26" "2016-01-27" "2016-01-28" "2016-01-29"
 [21] "2016-02-01" "2016-02-02" "2016-02-03" "2016-02-04" "2016-02-05"
 [26] "2016-02-10" "2016-02-11" "2016-02-12" "2016-02-15" "2016-02-16"
 [31] "2016-02-17" "2016-02-18" "2016-02-19" "2016-02-22" "2016-02-23"
 [36] "2016-02-24" "2016-02-25" "2016-02-26" "2016-02-29" "2016-03-01"
 [41] "2016-03-02" "2016-03-03" "2016-03-04" "2016-03-07" "2016-03-08"
 [46] "2016-03-09" "2016-03-10" "2016-03-11" "2016-03-14" "2016-03-15"
 [51] "2016-03-16" "2016-03-17" "2016-03-18" "2016-03-21" "2016-03-22"
 [56] "2016-03-23" "2016-03-24" "2016-03-28" "2016-03-29" "2016-03-30"
 [61] "2016-03-31" "2016-04-01" "2016-04-04" "2016-04-05" "2016-04-06"
 [66] "2016-04-07" "2016-04-08" "2016-04-11" "2016-04-12" "2016-04-13"
 [71] 

US,UK
2009-03-07 12:00:00,2009-03-07 20:00:00
2009-03-08 12:00:00,2009-03-08 19:00:00
2009-03-28 12:00:00,2009-03-28 19:00:00
2009-03-29 12:00:00,2009-03-29 20:00:00
2009-10-24 12:00:00,2009-10-24 20:00:00
2009-10-25 12:00:00,2009-10-25 19:00:00
2009-10-31 12:00:00,2009-10-31 19:00:00
2009-11-01 12:00:00,2009-11-01 20:00:00


# Multi-indices, stacking, and pivot tables

In [22]:
grouped = group_by(dt_joined, State_Code, County_Code)
grouped_summ = summarise(grouped,
                        sum_NUM_ALL = sum(NUM_ALL),
                        sum_NUM_FHA = sum(NUM_FHA))

head(grouped_summ)
library(reshape2)
unstacked = dcast(grouped_summ, State_Code ~ County_Code, value.var = "sum_NUM_FHA")
head(unstacked)

State_Code,County_Code,sum_NUM_ALL,sum_NUM_FHA
10,001,1508,328
10,003,4620,1716
10,005,2977,229
11,001,6605,1154
12,001,1542,439
12,003,99,27


Warning message:
"package 'reshape2' was built under R version 3.3.3"
Attaching package: 'reshape2'

The following objects are masked from 'package:data.table':

    dcast, melt



State_Code,001,003,005,006,007,009,011,013,015,...,760,770,775,790,800,810,820,830,840,NA
10,328,1716,229,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
11,1154,NA,NA,NA,NA,NA,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
12,439,27,241,NA,11,1514,5059,3,278,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
13,17,2,5,NA,1,30,16,209,199,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
15,109,259,NA,NA,40,163,NA,NA,NA,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
16,1426,3,220,NA,12,7,51,31,11,...,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
